Exercício 2

Enunciado do Problema

Use o package Cryptography para criar uma cifra com autenticação de meta-dados a partir de um PRG

    1. Criar um gerador pseudo-aleatório do tipo XOF (“extened output function”) usando o SHAKE256, para gerar uma sequência de palavras de 64 bits. 
        i. O gerador deve poder gerar até um limite de 2^n palavras (n é  um parâmetro) armazenados em long integers do Python.
        ii. A “seed” do gerador funciona como cipher_key} e é gerado por um KDF a partir de uma “password” .
        ii. A autenticação do criptograma e dos dados associados é feita usando o próprio SHAKE256.
    
    2. Defina os algoritmos de cifrar e decifrar : para cifrar/decifrar uma mensagem com blocos de 64 bits, os “outputs” do gerador são usados como máscaras XOR dos blocos da mensagem. 
    Essencialmente a cifra básica é uma implementação do “One Time Pad”.

Descrição do Problema

Abordagem

Código de resolução

In [ ]:
!pip install cryptography

O código implementado cria uma classe OneTimePad que implementa a cifra "One Time Pad" para cifrar e decifrar mensagens com blocos de 64 bits. A classe tem as seguintes funções implementadas:

_init_(self, password, n=128): é o construtor da classe. Recebe uma password e um parâmetro n (padrão 128) que é a quantidade de bits que serão gerados pelo gerador pseudo-aleatório XOF. Utiliza o algoritmo PBKDF2HMAC (Password-Based Key Derivation Function 2) para derivar uma chave criptográfica a partir da password, com um salt aleatório gerado pelo secrets.token_bytes(). A chave é armazenada na forma de um inteiro em self.key.

encrypt(self, message): recebe uma mensagem, que é cifrada usando a cifra "One Time Pad" com a chave armazenada em self.key. Retorna a mensagem cifrada como um objeto bytearray.

decrypt(self, masked_message): recebe uma mensagem cifrada, que é decifrada usando a cifra "One Time Pad" com a chave armazenada em self.key. Retorna a mensagem decifrada como um objeto bytearray.

_generate_mask(self, block_index): gera uma máscara para cifrar ou decifrar um bloco da mensagem. Recebe como entrada o índice do bloco em bytes e retorna uma máscara como um inteiro.

_generate_output(self, block_index): gera a sequência pseudo-aleatória de bits para cifrar ou decifrar um bloco da mensagem. Recebe como entrada o índice do bloco em bytes e retorna a sequência como um objeto bytearray.

_generate_data(self, block_index, data_index): gera os dados que serão usados como entrada para o XOF para gerar a sequência pseudo-aleatória de bits. Recebe como entrada o índice do bloco e o índice dos dados dentro do bloco,ambos em bytes, e retorna os dados como um objeto bytearray.


Além da classe e das funções implementadas, foram importadas as seguintes bibliotecas: 

PBKDF2HMAC do pacote cryptography.hazmat.primitives.kdf.pbkdf2: usado para derivar uma chave criptográfica a partir da password.

hashes do pacote cryptography.hazmat.primitives: usado para selecionar o algoritmo de hash SHA256 para o PBKDF2HMAC e para o XOF.

default_backend do pacote cryptography.hazmat.backends: usado para selecionar o backend padrão para as operações criptográficas.

secrets: usado para gerar o salt aleatório.

In [ ]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import secrets

class OneTimePad:
    def __init__(self, password, n=128):
        self.n = n
        salt = secrets.token_bytes(16)
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=self.n // 8,
            salt=salt,
            iterations=100000,
        )
        key = kdf.derive(password.encode())
        self.key = int.from_bytes(key, byteorder="big")

    def encrypt(self, message):
      padding_bytes = 8 - (len(message) % 8)
      message += bytes([padding_bytes] * padding_bytes)
      masked_message = bytearray()
      for i in range(0, len(message), 8):
          block = int.from_bytes(message[i:i+8], byteorder="big")
          mask = self._generate_mask(i // 8)
          masked_block = block ^ mask
          masked_message += masked_block.to_bytes(8, byteorder="big")
      return masked_message


    def decrypt(self, masked_message):
      message = bytearray()
      for i in range(0, len(masked_message), 8):
          masked_block = int.from_bytes(masked_message[i:i+8], byteorder="big")
          mask = self._generate_mask(i // 8)
          block = masked_block ^ mask
          message += block.to_bytes(8, byteorder="big")
      # Remove os bytes adicionais da mensagem original
      padding_bytes = message[-1]
      return message[:-padding_bytes]


    def _generate_mask(self, block_index):
        output = self._generate_output(block_index)
        mask = int.from_bytes(output[:8], byteorder="big")
        return mask

    def _generate_output(self, block_index):
        n_bytes = self.n // 8
        output = bytearray()
        for i in range(n_bytes):
            data = self._generate_data(block_index, i)
            output.append(data)
        return bytes(output)

    def _generate_data(self, block_index, data_index):
        data = self.key.to_bytes(16, byteorder="big")
        data += block_index.to_bytes(8, byteorder="big")
        data += data_index.to_bytes(8, byteorder="big")
        digest = hashes.Hash(hashes.SHAKE256(self.n), backend=default_backend())
        digest.update(data)
        return digest.finalize()[0]

Exemplos e testes de aplicação